In [223]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU, QuantIdentity
from brevitas.quant import IntBias
from brevitas.core.restrict_val import RestrictValueType
from brevitas.quant import Int8ActPerTensorFloat, Uint8ActPerTensorFloat, Int32Bias
from brevitas.quant import Int8WeightPerTensorFloat, Int8WeightPerChannelFloat
from brevitas.quant.scaled_int import Int32Bias
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score
import numpy as np
import time
import warnings

In [198]:
import brevitas
print(brevitas.__version__)

0.10.3


In [199]:
FIRST_LAYER_BIT_WIDTH = 8
MIDDLE_LAYER_BIT_WIDTH = 4

In [210]:
# For weights and activations
class CommonIntWeightPerTensorQuant(Int8WeightPerTensorFloat):
    scaling_min_val = 2e-16

class CommonIntWeightPerChannelQuant(Int8WeightPerChannelFloat):
    scaling_per_output_channel = True

class CommonIntActQuant(Int8ActPerTensorFloat):
    scaling_min_val = 2e-16
    restrict_scaling_type = RestrictValueType.LOG_FP

class CommonUintActQuant(Uint8ActPerTensorFloat):
    scaling_min_val = 2e-16
    restrict_scaling_type = RestrictValueType.LOG_FP

class CommonUint4ActQuant(CommonUintActQuant):
    bit_width = MIDDLE_LAYER_BIT_WIDTH

class CommonInt4ActQuant(CommonIntActQuant):
    bit_width = MIDDLE_LAYER_BIT_WIDTH

In [211]:
class QuantizedCustomMNISTNet(nn.Module):
    def __init__(self):
        super(QuantizedCustomMNISTNet, self).__init__()
        
        # First layer (8-bit)
        self.conv1 = QuantConv2d(
            in_channels=1,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=1,
            weight_bit_width=FIRST_LAYER_BIT_WIDTH,
            bias=True,
            bias_quant=None,
            weight_quant=CommonIntWeightPerChannelQuant,
            input_quant=None,
            output_quant=CommonUintActQuant,
            return_quant_tensor=True
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = QuantReLU(
            act_quant=CommonUintActQuant,
            bit_width=FIRST_LAYER_BIT_WIDTH,
            return_quant_tensor=True
        )
        
        # Middle layers (4-bit)
        self.conv2 = QuantConv2d(
            in_channels=64,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=1,
            weight_bit_width=MIDDLE_LAYER_BIT_WIDTH,
            bias=True,
            bias_quant=Int32Bias,
            weight_quant=CommonIntWeightPerChannelQuant,
            input_quant=CommonUint4ActQuant,
            output_quant=CommonUint4ActQuant,
            return_quant_tensor=True
        )
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = QuantReLU(
            act_quant=CommonInt4ActQuant,
            bit_width=MIDDLE_LAYER_BIT_WIDTH,
            return_quant_tensor=True
        )
        
        self.conv3 = QuantConv2d(
            in_channels=64,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=1,
            weight_bit_width=MIDDLE_LAYER_BIT_WIDTH,
            bias=True,
            bias_quant=Int32Bias,
            weight_quant=CommonIntWeightPerChannelQuant,
            input_quant=CommonUint4ActQuant,
            output_quant=CommonUint4ActQuant,
            return_quant_tensor=True
        )
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = QuantReLU(
            act_quant=CommonInt4ActQuant,
            bit_width=MIDDLE_LAYER_BIT_WIDTH,
            return_quant_tensor=True
        )
        
        # Fully connected layers (4-bit)
        self.fc1 = QuantLinear(
            3136, 128,
            bias=True,
            weight_bit_width=MIDDLE_LAYER_BIT_WIDTH,
            bias_quant=Int32Bias,
            weight_quant=CommonIntWeightPerTensorQuant,
            input_quant=CommonUint4ActQuant,
            output_quant=CommonUint4ActQuant,
            return_quant_tensor=True
        )
        self.relu4 = QuantReLU(
            act_quant=CommonInt4ActQuant,
            bit_width=MIDDLE_LAYER_BIT_WIDTH,
            return_quant_tensor=True
        )
        
        # Last layer (8-bit)
        self.fc2 = QuantLinear(
            128, 10,
            bias=True,
            weight_bit_width=FIRST_LAYER_BIT_WIDTH,
            bias_quant=IntBias,
            weight_quant=CommonIntWeightPerTensorQuant,
            input_quant=CommonUintActQuant,
            output_quant=CommonUintActQuant,
            return_quant_tensor=True
        )

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = self.relu2(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = self.relu3(self.bn3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [202]:
def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    return train_loader, test_loader

In [203]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [204]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')
    return accuracy

In [205]:
def save_onnx(model, device, filename="mnist_cnn_quantized.onnx"):
    model.eval()
    model.to(device)

    print("Checking module devices and bit widths:")
    for name, module in model.named_modules():
        if hasattr(module, 'weight'):
            print(f"{name} weight is on device: {module.weight.device}")
        if hasattr(module, 'bias') and module.bias is not None:
            print(f"{name} bias is on device: {module.bias.device}")
        if hasattr(module, 'quant_input'):
            print(f"{name} quant_input is on device: {module.quant_input.quant_proxy.device}")
        if hasattr(module, 'weight_quant'):
            print(f"{name} weight_quant is on device: {module.weight_quant.tensor_quant.device}")
        
        if hasattr(module, 'bit_width'):
            try:
                bit_width = module.bit_width()
                print(f"{name} bit_width: {bit_width}")
            except Exception as e:
                print(f"Error getting bit_width for {name}: {str(e)}")
        
        if hasattr(module, 'tracked_parameter_list'):
            print(f"{name} tracked parameters: {len(module.tracked_parameter_list)}")

    dummy_input = torch.randn(1, 1, 28, 28).to(device)

    try:
        export_onnx_qcdq(model, dummy_input, filename, 
                         input_names=['input'], output_names=['output'],
                         opset_version=13)
        print(f"Model saved in QCDQ ONNX format as {filename}")
    except Exception as e:
        print(f"Error during ONNX export: {str(e)}")
        print("Attempting to export without quantization...")
        try:
            torch.onnx.export(model, dummy_input, filename, 
                              input_names=['input'], output_names=['output'],
                              opset_version=13)
            print(f"Model saved in standard ONNX format as {filename}")
        except Exception as e2:
            print(f"Error during standard ONNX export: {str(e2)}")

In [206]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if not torch.cuda.is_available():
        print("CUDA is not available. Running on CPU.")
    else:
        print(f"Running on GPU: {torch.cuda.get_device_name(0)}")
    
    model = QuantizedCustomMNISTNet().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    train_loader, test_loader = load_data()
    
    print(model)
    
    best_accuracy = 0
    for epoch in range(1, 11):
        train(model, device, train_loader, optimizer, criterion, epoch)
        accuracy = test(model, device, test_loader, criterion)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_quantized_model.pth')
    
    print(f"Best test accuracy: {best_accuracy:.2f}%")
    
    # Load the best model and save in ONNX format
    model = QuantizedCustomMNISTNet().to(device)
    model.load_state_dict(torch.load('best_quantized_model.pth', map_location=device))
    model.eval()
    
    # save_onnx(model, device)

In [226]:
if __name__ == '__main__':
    main()

Running on GPU: NVIDIA GeForce RTX 3050 Laptop GPU
QuantizedCustomMNISTNet(
  (conv1): QuantConv2d(
    1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (output_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (fused_activation_quant_proxy): FusedActivationQuantProxy(
        (activation_impl): Identity()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClamp()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
          )
          (scaling_impl): ParameterFromRuntimeStatsScaling(
            (stats_input_view_shape_impl): OverTensorView()
            (stats): _Stats(
              (stats_impl): AbsPercentile()
            )
            (restrict_scaling): _RestrictValue(
              (re

In [227]:
# Load the saved model
model = QuantizedCustomMNISTNet()
model.load_state_dict(torch.load('best_quantized_model.pth', map_location=torch.device('cpu')))
model.eval()

# Function to print model architecture and bit widths
def print_model_info(model):
    print("Model Architecture and Bit Widths:")
    for name, module in model.named_modules():
        if isinstance(module, (QuantConv2d, QuantLinear, QuantReLU, QuantIdentity)):
            print(f"\nLayer: {name}")
            print(f"Type: {type(module).__name__}")
            
            if hasattr(module, 'weight_bit_width'):
                print(f"Weight Bit Width: {module.weight_bit_width}")
            
            if hasattr(module, 'bias') and module.bias is not None:
                print(f"Has Bias: Yes")
            else:
                print(f"Has Bias: No")
            
            if hasattr(module, 'input_quant'):
                print(f"Input Quant: {type(module.input_quant).__name__}")
            
            if hasattr(module, 'output_quant'):
                print(f"Output Quant: {type(module.output_quant).__name__}")
            
            if isinstance(module, (QuantConv2d, QuantLinear)):
                print(f"In Features: {module.in_channels if isinstance(module, QuantConv2d) else module.in_features}")
                print(f"Out Features: {module.out_channels if isinstance(module, QuantConv2d) else module.out_features}")
            
            if isinstance(module, QuantConv2d):
                print(f"Kernel Size: {module.kernel_size}")
                print(f"Stride: {module.stride}")
                print(f"Padding: {module.padding}")

# Print model information
print_model_info(model)

Model Architecture and Bit Widths:

Layer: conv1
Type: QuantConv2d
Has Bias: Yes
Input Quant: ActQuantProxyFromInjector
Output Quant: ActQuantProxyFromInjector
In Features: 1
Out Features: 64
Kernel Size: (3, 3)
Stride: (1, 1)
Padding: (1, 1)

Layer: relu1
Type: QuantReLU
Has Bias: No
Input Quant: ActQuantProxyFromInjector

Layer: conv2
Type: QuantConv2d
Has Bias: Yes
Input Quant: ActQuantProxyFromInjector
Output Quant: ActQuantProxyFromInjector
In Features: 64
Out Features: 64
Kernel Size: (3, 3)
Stride: (1, 1)
Padding: (1, 1)

Layer: relu2
Type: QuantReLU
Has Bias: No
Input Quant: ActQuantProxyFromInjector

Layer: conv3
Type: QuantConv2d
Has Bias: Yes
Input Quant: ActQuantProxyFromInjector
Output Quant: ActQuantProxyFromInjector
In Features: 64
Out Features: 64
Kernel Size: (3, 3)
Stride: (1, 1)
Padding: (1, 1)

Layer: relu3
Type: QuantReLU
Has Bias: No
Input Quant: ActQuantProxyFromInjector

Layer: fc1
Type: QuantLinear
Has Bias: Yes
Input Quant: ActQuantProxyFromInjector
Output Qua

In [231]:
# Load the state dict and convert to ONNX
def convert_state_dict_to_onnx(state_dict_path, onnx_path):
    # Create an instance of the model
    model = QuantizedCustomMNISTNet()
    
    # Load the state dict
    model.load_state_dict(torch.load(state_dict_path, map_location=torch.device('cpu')))
    
    # Set the model to evaluation mode
    model.eval()
    
    # Create a dummy input tensor
    dummy_input = torch.randn(1, 1, 28, 28)
    
    # Export the model to ONNX
    try:
        export_onnx_qcdq(model, dummy_input, onnx_path, 
                         input_names=['input'], output_names=['output'],
                         opset_version=13)
        print(f"Model successfully exported to {onnx_path}")
    except Exception as e:
        print(f"Error during ONNX export: {str(e)}")
        print("Attempting to export without quantization...")
        try:
            torch.onnx.export(model, dummy_input, onnx_path, 
                              input_names=['input'], output_names=['output'],
                              opset_version=13)
            print(f"Model exported in standard ONNX format to {onnx_path}")
        except Exception as e2:
            print(f"Error during standard ONNX export: {str(e2)}")

# Use the function
state_dict_path = 'best_quantized_model.pth'
onnx_path = 'quantized_mnist_model.onnx'
convert_state_dict_to_onnx(state_dict_path, onnx_path)

Error during ONNX export: '>' not supported between instances of 'NoneType' and 'float'
Attempting to export without quantization...


/home/muhammadzaky/.local/lib/python3.10/site-packages/brevitas/quant_tensor/__init__.py:68: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  training = torch.tensor(training, dtype=torch.bool)
/home/muhammadzaky/.local/lib/python3.10/site-packages/brevitas/export/common/handler/qcdq.py:52: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert bools
/home/muhammadzaky/.local/lib/python3.10/site-packages/brevitas/quant_tensor/__init__.py:66: TracerWarning: torch.tensor results are registered as constants in the trace. Y

Model exported in standard ONNX format to quantized_mnist_model.onnx


In [228]:
def load_model(model_path):
    model = QuantizedCustomMNISTNet()
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def inference(model, input_tensor):
    with torch.no_grad():
        output = model(input_tensor)
    return output

# Example usage
if __name__ == "__main__":
    # Load the model
    model_path = 'best_quantized_model.pth'
    model = load_model(model_path)

    # Create a dummy input (replace this with your actual input data)
    dummy_input = torch.randn(1, 1, 28, 28)  # Batch size 1, 1 channel, 28x28 image

    # Perform inference
    output = inference(model, dummy_input)

    # Process the output
    predicted_class = output.argmax(dim=1).item()
    print(f"Predicted class: {predicted_class}")
    print(f"Output probabilities: {torch.exp(output)}")

Predicted class: 3
Output probabilities: tensor([[0.0080, 0.0445, 0.0445, 0.5999, 0.0080, 0.2282, 0.0126, 0.0383, 0.0080,
         0.0080]])


In [229]:
import torch
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU, QuantIdentity

def inspect_quantization(model):
    print("Quantization Inspection:")
    print("-----------------------")

    for name, module in model.named_modules():
        if isinstance(module, (QuantConv2d, QuantLinear, QuantReLU, QuantIdentity)):
            print(f"\nLayer: {name}")
            print(f"Type: {type(module).__name__}")
            
            # Check weight quantization
            if hasattr(module, 'weight'):
                weight = module.weight
                print(f"Weight shape: {weight.shape}")
                print(f"Weight dtype: {weight.dtype}")
                print(f"Weight range: [{weight.min().item():.4f}, {weight.max().item():.4f}]")
                print(f"Weight unique values: {torch.unique(weight).numel()}")
                
                if hasattr(module.weight_quant, 'bit_width'):
                    bit_width = module.weight_quant.bit_width
                    if callable(bit_width):
                        bit_width = bit_width()
                    print(f"Weight bit width: {bit_width}")
                else:
                    print("Weight bit width not found")
                
                if hasattr(module, 'weight_quant') and hasattr(module.weight_quant, 'scaling_impl'):
                    scaling_factor = module.weight_quant.scaling_impl.scaling_factor
                    if callable(scaling_factor):
                        scaling_factor = scaling_factor()
                    print(f"Weight scaling factor: {scaling_factor}")

            # Check bias quantization
            if hasattr(module, 'bias') and module.bias is not None:
                bias = module.bias
                print(f"Bias shape: {bias.shape}")
                print(f"Bias dtype: {bias.dtype}")
                print(f"Bias range: [{bias.min().item():.4f}, {bias.max().item():.4f}]")
                print(f"Bias unique values: {torch.unique(bias).numel()}")

            # Check activation quantization
            if hasattr(module, 'act_quant'):
                print(f"Activation quantization: {type(module.act_quant).__name__}")
                if hasattr(module.act_quant, 'bit_width'):
                    bit_width = module.act_quant.bit_width
                    if callable(bit_width):
                        bit_width = bit_width()
                    print(f"Activation bit width: {bit_width}")
                if hasattr(module.act_quant, 'scaling_impl'):
                    scaling_factor = module.act_quant.scaling_impl.scaling_factor
                    if callable(scaling_factor):
                        scaling_factor = scaling_factor()
                    print(f"Activation scaling factor: {scaling_factor}")

            # Check input quantization
            if hasattr(module, 'input_quant'):
                print(f"Input quantization: {type(module.input_quant).__name__}")
                if hasattr(module.input_quant, 'bit_width'):
                    bit_width = module.input_quant.bit_width
                    if callable(bit_width):
                        bit_width = bit_width()
                    print(f"Input bit width: {bit_width}")

            # Check output quantization
            if hasattr(module, 'output_quant'):
                print(f"Output quantization: {type(module.output_quant).__name__}")
                if hasattr(module.output_quant, 'bit_width'):
                    bit_width = module.output_quant.bit_width
                    if callable(bit_width):
                        bit_width = bit_width()
                    print(f"Output bit width: {bit_width}")

    print("\nOverall Model Statistics:")
    print("-------------------------")
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params}")
    
    quant_params = sum(p.numel() for name, p in model.named_parameters() if 'weight' in name or 'bias' in name)
    print(f"Quantized parameters: {quant_params}")
    
    print(f"Percentage of quantized parameters: {quant_params/total_params*100:.2f}%")

# Example usage
if __name__ == "__main__":
    model_path = 'best_quantized_model.pth'
    model = load_model(model_path)
    inspect_quantization(model)

Quantization Inspection:
-----------------------

Layer: conv1
Type: QuantConv2d
Weight shape: torch.Size([64, 1, 3, 3])
Weight dtype: torch.float32
Weight range: [-0.4256, 0.3891]
Weight unique values: 576
Weight bit width: 8.0
Bias shape: torch.Size([64])
Bias dtype: torch.float32
Bias range: [-0.3680, 0.3238]
Bias unique values: 64
Input quantization: ActQuantProxyFromInjector
Input bit width: None
Output quantization: ActQuantProxyFromInjector
Output bit width: 8.0

Layer: relu1
Type: QuantReLU
Activation quantization: ActQuantProxyFromInjector
Activation bit width: 8.0
Input quantization: ActQuantProxyFromInjector
Input bit width: None

Layer: conv2
Type: QuantConv2d
Weight shape: torch.Size([64, 64, 3, 3])
Weight dtype: torch.float32
Weight range: [-0.1213, 0.1581]
Weight unique values: 36852
Weight bit width: 4.0
Bias shape: torch.Size([64])
Bias dtype: torch.float32
Bias range: [-0.0495, 0.0386]
Bias unique values: 64
Input quantization: ActQuantProxyFromInjector
Input bit widt

In [230]:
def evaluate_mnist_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    all_probs = []
    correct = 0
    total = 0
    inference_times = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            start_time = time.time()
            output = model(data)
            inference_time = time.time() - start_time
            inference_times.append(inference_time)

            probs = F.softmax(output, dim=1)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = 100 * correct / total
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    all_probs = np.array(all_probs)

    # Confusion Matrix
    conf_matrix = confusion_matrix(all_targets, all_preds)

    # Precision, Recall, F1-Score
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)

    # ROC AUC Score (one-vs-all)
    roc_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr', average='weighted')

    # Top-3 Accuracy
    top3_preds = np.argsort(all_probs, axis=1)[:, -3:]
    top3_correct = np.any(top3_preds == all_targets.reshape(-1, 1), axis=1).sum()
    top3_accuracy = 100 * top3_correct / total

    # Inference Time
    avg_inference_time = sum(inference_times) / len(inference_times)

    # Model Size
    model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)

    results = {
        'Accuracy': accuracy,
        'Confusion Matrix': conf_matrix,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC AUC Score': roc_auc,
        'Top-3 Accuracy': top3_accuracy,
        'Average Inference Time (s)': avg_inference_time,
        'Model Size (parameters)': model_size
    }

    return results

# Usage
# Main execution
if __name__ == "__main__":
    model_path = 'best_quantized_model.pth'
    batch_size = 64
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the model
    model = load_model(model_path)
    model.to(device)

    # Load the data
    train_loader, test_loader = load_data(batch_size)

    # Evaluate the model
    results = evaluate_mnist_model(model, test_loader, device)
    
    # Print results
    for metric, value in results.items():
        print(f"{metric}:")
        print(value)
        print()

Accuracy:
99.38

Confusion Matrix:
[[ 978    0    0    0    0    0    1    1    0    0]
 [   0 1133    0    0    0    0    0    2    0    0]
 [   1    1 1024    0    0    0    2    4    0    0]
 [   0    0    0 1007    0    2    0    0    1    0]
 [   0    0    0    0  981    0    0    0    0    1]
 [   1    0    0    5    0  884    1    0    0    1]
 [   6    1    0    0    1    0  949    0    1    0]
 [   0    3    2    0    1    0    0 1021    1    0]
 [   2    0    2    0    0    0    0    0  968    2]
 [   1    0    0    1    8    3    0    2    1  993]]

Precision:
0.9938129545880825

Recall:
0.9938

F1-Score:
0.9937969300974159

ROC AUC Score:
0.9999712560098378

Top-3 Accuracy:
99.95

Average Inference Time (s):
0.013727355003356933

Model Size (parameters):
477719

